In [2]:
import torch as torch

c:\Users\xiang\anaconda3\envs\a4\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
a = torch.randn(2, 3)
b = torch.randn(2, 3, 8)

c = torch.bmm(a.unsqueeze(1), b).squeeze(1)
print(c.shape)

assert c.shape == (2, 8)

torch.Size([2, 8])


In [4]:
torch.cuda.is_available()

True